In [0]:
!unzip images.zip

In [0]:
from dataloader import HolonomicDataset

In [0]:
csv_file = 'data.csv'
grayscale = True
full_dataset = HolonomicDataset(csv_file,'',grayscale=grayscale)

In [0]:
from torch.utils.data import DataLoader
trainloader = DataLoader(full_dataset, batch_size=1, shuffle=False)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

class FirstNet(nn.Module):
  def __init__(self):
    super(FirstNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 8, 3, padding=1) 
    self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
    self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
    self.conv4 = nn.Conv2d(32, 64, 3, padding=1)
    #We need to introduce dropout for stochasticity
    self.fc1 = nn.Linear(8 * 8 * 64 + 4, 1024)
    self.fc2 = nn.Linear(1024, 512)
    self.fc3 = nn.Linear(512, 256)
    self.fc4 = nn.Linear(256, 128)
    self.fc5 = nn.Linear(128, 8)

  def forward(self, input):
    x=input[1]
    x=torch.unsqueeze(x,0).float()
    x = F.max_pool2d(F.relu(self.conv1(x)), 2) # 64x64
    x = F.max_pool2d(F.relu(self.conv2(x)), 2) # 32x32
    x = F.max_pool2d(F.relu(self.conv3(x)), 2) # 16x16
    x = F.max_pool2d(F.relu(self.conv4(x)), 2) # 8X8

    x = x.view(-1, self.num_flat_features(x))
    #X = torch.cat((torch.transpose(x,0,1), torch.transpose(input[0].float(),0,1), 0))
    x=torch.squeeze(x)
    input[0]=torch.squeeze(input[0])
    X = torch.cat((x,input[0].float()),0)
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = F.relu(self.fc3(X))
    X = F.relu(self.fc4(X))
    X = torch.sigmoid(self.fc5(X))
    return X

  def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features



In [0]:
#instance of the Conv Net
net = FirstNet().float()
learning_rate=0.0001
num_epochs=2
batch_size=1
#loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [51]:
losses = [];
# for epoch in range(num_epochs):
for i, (input_data, labels) in enumerate(trainloader):

    optimizer.zero_grad()
    outputs = torch.unsqueeze(net(input_data), 0)
    labels -= 1 # shift labels from 1-8 to 0-7 (fix off by one)
    loss = criterion(outputs, labels) # turns out one-hot encoding is not necessary
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (i+1) % 100 == 0:
        print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                %(epoch+1, num_epochs, i+1, len(full_dataset)//batch_size, loss.item()))

Epoch : 1/2, Iter : 100/10719,  Loss: 1.4368


1. Softmax/ Sigmoid Activation at the final layer
2. One hot encoding for the lebels